In [ ]:
library(tidyverse)
library(dplyr)
library(stringr)
library(microViz)
library(metagMisc)
library(phyloseq)
library(phylosignal)
library(ggplot2)
library(ggsci)
library(viridis)

#Muehlbauer results

###Import kraken results - Muehlbauer

In [ ]:
kraken.muehl=read_tsv("/ebio/abt3_scratch/aborbon/taxonomy_profiles/muehl/llmgp/output_muehl/kraken/all-combined-bracken.tsv",col_names=TRUE)
kraken.muehl

kraken.muehl.frac=select(kraken.muehl,name,taxonomy_id,taxonomy,ends_with("frac"))
colnames(kraken.muehl.frac)=gsub("_frac","",colnames(kraken.muehl.frac))
kraken.muehl.frac

###Create OTU table from kraken results

In [ ]:
#otu counts
kraken_muehl_otus=kraken.muehl.frac %>%
  select(-c(taxonomy_id,taxonomy))

otus_kraken.muehl=otu_table(kraken_muehl_otus[,2:ncol(kraken_muehl_otus)],taxa_are_rows = TRUE)
rownames(otus_kraken.muehl)=kraken_muehl_otus$name
otus_kraken.muehl

#taxonomy table
tax.kraken.muehl=kraken.muehl %>%
  select(name,taxonomy)

tax.kraken.muehl=tax_table(as.matrix(tax.kraken.muehl[,2:ncol(tax.kraken.muehl)]))
rownames(tax.kraken.muehl)=kraken.muehl$name


#metadata
mapping_ps_muehl=sample_data(filter(mappingAll,Dataset=="Muehlbauer_2020"))
rownames(mapping_ps_muehl)=(filter(mappingAll,Dataset=="Muehlbauer_2020"))$Sample

#Phyloseq kraken amato
ps.kraken.muehl=phyloseq(otus_kraken.muehl,tax.kraken.muehl,mapping_ps_muehl)
ps.kraken.muehl

In [ ]:

#Filter based on prevalence
filter_taxa_All.muehl = phyloseq_filter_prevalence(ps.kraken.muehl, prev.trh = 0.05, abund.trh = NULL,
  threshold_condition = "OR", abund.type = "total") #943 taxa and 809 samples

#Prune samples without any OTU
filter_taxa_All2.muehl=prune_samples(sample_sums(filter_taxa_All.muehl)>0,filter_taxa_All.muehl)
filter_taxa_All2.muehl

#Normalize relative abundances
filt.ps.relab.normAll.muehl  = transform_sample_counts(filter_taxa_All2.muehl, function(x) (x / sum(x)) ) 
filt.ps.relab.normAll.muehl

In [ ]:
diss.jacc.muehl=phyloseq_group_dissimilarity(filt.ps.relab.normAll.muehl,group="family",method="jaccard",between_groups = FALSE)+ggtitle("Jaccard")

diss.bray.muehl=phyloseq_group_dissimilarity(filt.ps.relab.normAll.muehl,group="family",method="bray",between_groups = FALSE)+ggtitle("Bray-Curtis")
# diss.unifrac1=phyloseq_group_dissimilarity(filt.ps.relab.normAll.vert,group="biome_1",method="unifrac",between_groups = FALSE)+ggtitle("uUniFrac")
# diss.wunifrac1=phyloseq_group_dissimilarity(filt.ps.relab.normAll.vert,group="biome_1",method="wunifrac",between_groups = FALSE)+ggtitle("wUniFrac")

diss.jacc.muehl
diss.bray.muehl

#Ordination plots with non-phylogenetic metrics

In [ ]:
plot_ordination(physeq = filt.ps.relab.normAll.muehl,ordinate(filt.ps.relab.normAll.muehl,distance="jaccard",method="PCoA"),type="samples",color="family")+
  theme_bw()+
  geom_point(size=2)+
  scale_color_lancet()+
  ggtitle("Jaccard")

plot_ordination(physeq = filt.ps.relab.normAll.muehl,ordinate(filt.ps.relab.normAll.amato,distance="bray",method="PCoA"),type="samples",color="family")+
  theme_bw()+
  geom_point(size=2)+
  scale_color_lancet()+
  ggtitle("BrayCurtis")

#Mantel test for Amato results

In [ ]:
#Subset results of flagellome for Muehlbauer dataset
ps_fla.muehl=ps_filter(filt.ps.relab.normAll, Dataset == "Muehlbauer_2020")
ps_fla.muehl

In [ ]:
#Match samples from flagellin results amd overal microbiome results
tmp.sam.muehl=ps_fla.muehl@sam_data
ps.filtab.muehl=phyloseq(filt.ps.relab.normAll.muehl@otu_table,filt.ps.relab.normAll.muehl@tax_table,tmp.sam.muehl)

#Distance matrices
bray.muehl.fla=distance(ps_fla.muehl,method="bray")
bray.muehl.ubiom=distance(ps.filtab.muehl,method="bray")

mantel.muehl=mantel(bray.muehl.fla,bray.muehl.ubiom,method="spearman")
mantel.muehl

#Overall microbiome compositon in Amato dataset

In [ ]:
ps.filtab.muehl

plot_ordination(physeq = ps.filtab.muehl,ordinate(ps.filtab.muehl,distance="bray",method="PCoA"),type="samples",color="family")+
  theme_bw()+
  geom_point(size=2)+
  scale_color_lancet()+
  ggtitle("BrayCurtis")

phyloseq_group_dissimilarity(ps.filtab.muehl,group="family",method="bray",between_groups = FALSE)+ggtitle("Bray-Curtis")

#Procrustes analysis

In [ ]:
typeof(otus_kraken.muehl)
estimate_richness(ps_fla.muehl)

In [ ]:
kraken_muehl_otus
otus_kraken.muehl

kraken.muehl.shannon$alpha <- diversity (kraken_muehl_otus[,kraken_muehl_otus$name],MARGIN=2,index="shannon")
otus_kraken.muehl
